In [1]:
import pandas as pd
import csv
import os
import json
import codecs
import io
import sys
import re

# ==========================================
# 1. INPUT HANDLING
# ==========================================
def get_user_file():
    print("\n" + "="*50)
    print("[STEP 1] UPLOAD TARGET CSV")
    print("="*50)
    try:
        from google.colab import files
        print("[INFO] Environment: Google Colab detected.")
        print("[ACTION] Please upload your .csv file now...")
        uploaded = files.upload()
        if not uploaded: return None
        return list(uploaded.keys())[0]
    except ImportError:
        print("[INFO] Environment: Local Python detected.")
        filepath = input("[INPUT] Enter path to .csv file: ").strip()
        return filepath if os.path.exists(filepath) else None

# ==========================================
# 2. INTELLIGENT PARSER ENGINE
# ==========================================
def extract_table_structure(filepath):
    print(f"\n[INFO] Analyzing table structure for: {filepath}...")

    # A. Detect Encoding (Crucial for Excel/Legacy files)
    encoding = detect_encoding(filepath)
    print(f"[INFO] Detected Encoding: {encoding}")

    # B. Heuristic Delimiter Detection & Row Skipping
    delimiter, skip_rows = detect_dialect(filepath, encoding)
    print(f"[INFO] Detected Delimiter: '{delimiter}' (Skip Rows: {skip_rows})")

    # C. Max Column Detection (Handle Broken/Ragged Rows)
    max_cols = detect_max_columns(filepath, encoding, delimiter, skip_rows)
    print(f"[INFO] Max Columns Detected: {max_cols}")

    try:
        # Generate generic column names to force alignment
        col_names = [f"Col_{i}" for i in range(max_cols)]

        # D. Robust Extraction
        # dtype=str ensures we don't crash on mixed types (e.g., column has "123" and "Error")
        df = pd.read_csv(
            filepath,
            sep=delimiter,
            encoding=encoding,
            skiprows=skip_rows,
            header=None,
            names=col_names,
            engine='python',
            dtype=str,
            on_bad_lines='skip'
        )

        # E. Header Repair (Handle Messy Headers)
        df = repair_header(df, max_cols)

        # F. Advanced Cleaning
        # 1. Strip whitespace from all string cells
        df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        # 2. Drop empty rows/cols
        df.dropna(how='all', inplace=True)
        df.dropna(axis=1, how='all', inplace=True)

        # 3. Handle Duplicate Rows (Messy Repeats)
        initial_rows = len(df)
        df.drop_duplicates(inplace=True)
        dropped_rows = initial_rows - len(df)
        if dropped_rows > 0:
            print(f"[INFO] Removed {dropped_rows} duplicate rows.")

        return df

    except Exception as e:
        print(f"[ERROR] Parsing Failed: {e}")
        return None

def repair_header(df, max_cols):
    """
    Promotes the first row to header, fixing duplicate names (e.g., 'Date', 'Date' -> 'Date', 'Date_1')
    """
    if df.empty: return df

    # Get first row values
    first_row = df.iloc[0].astype(str).tolist()

    # Generate unique column names
    seen_cols = {}
    new_columns = []

    for i in range(max_cols):
        # If we are within the first row's data range
        if i < len(first_row):
            col_name = first_row[i].strip()
            # Handle empty header cells
            if col_name in ['nan', 'None', '', 'null']:
                col_name = f"Unknown_Col_{i}"
        else:
            col_name = f"Extra_Data_{i}"

        # Deduplicate
        if col_name in seen_cols:
            seen_cols[col_name] += 1
            new_columns.append(f"{col_name}_{seen_cols[col_name]}")
        else:
            seen_cols[col_name] = 0
            new_columns.append(col_name)

    df.columns = new_columns
    df = df.iloc[1:].reset_index(drop=True)
    return df

def detect_encoding(filepath):
    """Tries various encodings to find the one that doesn't crash."""
    codings = ['utf-8', 'cp1252', 'latin-1', 'utf-16', 'iso-8859-1']
    for enc in codings:
        try:
            with codecs.open(filepath, 'r', encoding=enc) as f:
                f.read(4096)
            return enc
        except UnicodeDecodeError:
            continue
    return 'utf-8'

def detect_dialect(filepath, encoding):
    """Uses Sniffer to guess delimiter and skip metadata rows."""
    with codecs.open(filepath, 'r', encoding=encoding, errors='replace') as f:
        sample_lines = [f.readline() for _ in range(20)] # Read more lines for better accuracy

    # Filter out empty lines for analysis
    non_empty_lines = [line for line in sample_lines if line.strip()]
    sample_text = "".join(non_empty_lines)

    try:
        # Heuristic: Allow specialized delimiters if standard ones fail
        dialect = csv.Sniffer().sniff(sample_text, delimiters=[',', ';', '\t', '|', ':'])
        delimiter = dialect.delimiter
    except:
        if not non_empty_lines: return ',', 0
        # Manual fallback
        first_line = non_empty_lines[0]
        counts = {d: first_line.count(d) for d in [',', ';', '\t', '|']}
        delimiter = max(counts, key=counts.get)

    # Detect Metadata Header Garbage (Lines with significantly fewer delimiters)
    skip_rows = 0
    delimiter_counts = [line.count(delimiter) for line in non_empty_lines]

    if len(delimiter_counts) > 2:
        most_common_cols = max(set(delimiter_counts), key=delimiter_counts.count)
        for i, count in enumerate(delimiter_counts):
            # If a line has the "standard" number of columns, stop skipping
            if count >= most_common_cols:
                break
            skip_rows += 1

    return delimiter, skip_rows

def detect_max_columns(filepath, encoding, delimiter, skip_rows):
    """
    Scans the entire file to find the row with the MOST separators.
    This handles 'Ragged CSVs' where a row in the middle is wider than the header.
    """
    max_sep_count = 0
    try:
        with codecs.open(filepath, 'r', encoding=encoding, errors='ignore') as f:
            # Skip metadata
            for _ in range(skip_rows):
                next(f, None)

            for line in f:
                if not line.strip(): continue
                # Count separators (naive but effective for sizing)
                count = line.count(delimiter) + 1
                if count > max_sep_count:
                    max_sep_count = count
        return max_sep_count
    except:
        return 1 # Fallback

# ==========================================
# 3. REPORTING & VISUALIZATION
# ==========================================
def generate_report(df):
    if df is None: return

    print("\n" + "="*50)
    print("[RESULT] EXACT TABLE EXTRACTION REPORT")
    print("="*50)

    rows, cols = df.shape
    print(f"✅ Status:        SUCCESS")
    print(f"📊 Dimensions:    {rows} Rows x {cols} Columns")

    # Configure Pandas to show EVERYTHING (Real Layout)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', 50)

    print("-" * 50)
    print("ASCII Table Layout (Grid View - First 15 Rows):")
    try:
        print(df.head(15).to_markdown(index=False, tablefmt="grid"))
    except ImportError:
        print(df.head(15).to_string(index=False))
        print("\n[TIP] Install 'tabulate' for prettier grids: pip install tabulate")

    if rows > 15:
        print(f"\n... ({rows - 15} more rows hidden)")

    print("-" * 50)
    print("JSON Data Structure (First Record):")
    print(df.head(1).to_json(orient='records', indent=4))

# ==========================================
# 4. MAIN
# ==========================================
if __name__ == "__main__":
    target = get_user_file()
    if target:
        structured_df = extract_table_structure(target)
        generate_report(structured_df)


[STEP 1] UPLOAD TARGET CSV
[INFO] Environment: Google Colab detected.
[ACTION] Please upload your .csv file now...


Saving WELL.csv to WELL.csv

[INFO] Analyzing table structure for: WELL.csv...
[INFO] Detected Encoding: utf-8
[INFO] Detected Delimiter: ',' (Skip Rows: 0)
[INFO] Max Columns Detected: 55

[RESULT] EXACT TABLE EXTRACTION REPORT
✅ Status:        SUCCESS
📊 Dimensions:    13903 Rows x 45 Columns
--------------------------------------------------
ASCII Table Layout (Grid View - First 15 Rows):
+------+----------------+-------------+-------------------+------------+-----------+------------+--------+---------+-------------+----------------+--------------------+------------+-------------+---------+---------------+-------------+-----------+--------------------+-------------+--------------+------------+------------+----------------+------------+----------------+----------------+------------------+----------------+------------------+--------+------------+-----------+-------------+--------------+-----------+----------------+------------------+------------+--------------+----------------------+--